In [33]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [34]:
!pip freeze | grep python

fastjsonschema @ file:///tmp/build/80754af9/python-fastjsonschema_1620414857593/work/dist
ipython @ file:///tmp/build/80754af9/ipython_1648817057602/work
ipython-genutils @ file:///tmp/build/80754af9/ipython_genutils_1606773439826/work
libarchive-c @ file:///tmp/build/80754af9/python-libarchive-c_1617780486945/work
msgpack @ file:///tmp/build/80754af9/msgpack-python_1612287166301/work
python-dateutil @ file:///tmp/build/80754af9/python-dateutil_1626374649649/work
python-lsp-black @ file:///tmp/build/80754af9/python-lsp-black_1634232156041/work
python-lsp-jsonrpc==1.0.0
python-lsp-server==1.2.4
python-slugify @ file:///tmp/build/80754af9/python-slugify_1620405669636/work
python-snappy @ file:///tmp/build/80754af9/python-snappy_1610133040135/work


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [14]:
year = 2021
month = 2
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [15]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [16]:
import numpy as np
np.mean(y_pred)

16.191691679979066

In [17]:
df['ride_id'] = f'{year:04d}/{month:02d}_'+df.index.astype('str')

In [24]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred

In [27]:
output_file = f'{year:04d}-{month:02d}_'+'results.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [32]:
import os
os.path.getsize(output_file)/(1024**2)

18.798294067382812